In [6]:
import os
import numpy as np
import pandas as pd
import warnings

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Flatten
from tensorflow.compat.v1.keras.layers import TimeDistributed
from tensorflow.keras.layers import Conv1D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import ConvLSTM2D

# This is not recommended but I am doing this to suppress warnings from SARIMAX
warnings.simplefilter('ignore')

countryName = 'US'
nFeatures = 1

nDaysMin = 10

nValid = 10
nTest = 10

dataDir = os.path.join('data', 'JHU', 'upto07082020_forPublication')


# confirmedFilename = 'time_series_covid19_confirmed_global.csv'
# deathsFilename = 'time_series_covid19_deaths_global.csv'
# recoveredFilename = 'time_series_covid19_recovered_global.csv'

confirmedFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_confirmed_global.csv'
deathsFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_deaths_global.csv'
recoveredFilename = 'https://raw.githubusercontent.com/arkobarman/covid-19_timeSeriesAnalysis/master/data/JHU/upto07082020_forPublication/time_series_covid19_recovered_global.csv'

In [7]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
    X, y = list(), list()
    for i in range(len(sequence)):
        # find the end of this pattern
        end_ix = i + n_steps
        # check if we are beyond the sequence
        if end_ix > len(sequence)-1:
            break
        # gather input and output parts of the pattern
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

In [8]:
def meanAbsolutePercentageError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    absPcErrorList = [absError/yTrue for absError, yTrue in zip(absErrorList, yTrueList)]
    MAPE = 100*np.mean(absPcErrorList)
    return MAPE

def meanForecastError(yTrueList, yPredList):
    forecastErrors = [yTrue - yPred for yTrue, yPred in zip(yTrueList, yPredList)]
    MFE = np.mean(forecastErrors)
    return MFE

def meanAbsoluteError(yTrueList, yPredList):
    absErrorList = [np.abs(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(absErrorList)

def meanSquaredError(yTrueList, yPredList):
    sqErrorList = [np.square(yTrue - yPred) for yTrue, yPred in zip(yTrueList, yPredList)]
    return np.mean(sqErrorList)

def rootMeanSquaredError(yTrueList, yPredList):
    return np.sqrt(meanSquaredError(yTrueList, yPredList))

def medianSymmetricAccuracy(yTrueList, yPredList):
    '''https://helda.helsinki.fi//bitstream/handle/10138/312261/2017SW001669.pdf?sequence=1'''
    logAccRatioList = [np.abs(np.log(yPred/yTrue)) for yTrue, yPred in zip(yTrueList, yPredList)]
    MdSA = 100*(np.exp(np.median(logAccRatioList))-1)
    return MdSA

In [9]:
# Function to get all three frames for a given country
def getCountryCovidFrDict(countryName):
    countryCovidFrDict = {}
    for key in covidFrDict.keys():
        dataFr = covidFrDict[key]
        countryCovidFrDict[key] = dataFr[dataFr['Country/Region'] == countryName]
    return countryCovidFrDict

In [10]:
# Load all 3 csv files
covidFrDict = {}
# covidFrDict['confirmed'] = pd.read_csv(os.path.join(dataDir, confirmedFilename))
# covidFrDict['deaths'] = pd.read_csv(os.path.join(dataDir, deathsFilename))
covidFrDict['confirmed'] = pd.read_csv(confirmedFilename)
covidFrDict['deaths'] = pd.read_csv(deathsFilename)

# Recovered is back again!
covidFrDict['recovered'] = pd.read_csv(recoveredFilename)

countryCovidFrDict = getCountryCovidFrDict(countryName)

# Get list of dates
colNamesList = list(countryCovidFrDict['confirmed'])
dateList = [colName for colName in colNamesList if '/20' in colName]
dataList = [countryCovidFrDict['confirmed'][date].iloc[0] for date in dateList]
dataDict = dict(zip(dateList, dataList))

# Get time series for cases > 100 only
daysSince = 100
nCasesGreaterDaysSinceList = []
datesGreaterDaysSinceList = []

for key in dataDict.keys():
    if dataDict[key] > daysSince:
        datesGreaterDaysSinceList.append(key)
        nCasesGreaterDaysSinceList.append(dataDict[key])
        
XList, yList = split_sequence(nCasesGreaterDaysSinceList, nDaysMin)

XTrainList = XList[0:len(XList)-(nValid + nTest)]
XValidList = XList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
XTestList = XList[-nTest:]

yTrain = yList[0:len(XList)-(nValid + nTest)]
yValid = yList[len(XList)-(nValid+nTest):len(XList)-(nTest)]
yTest = yList[-nTest:]

print('Total size of data points for LSTM:', len(yList))
print('Size of training set:', len(yTrain))
print('Size of validation set:', len(yValid))
print('Size of test set:', len(yTest))

# Convert from list to matrix
XTrain = XTrainList.reshape((XTrainList.shape[0], XTrainList.shape[1], nFeatures))
XValid = XValidList.reshape((XValidList.shape[0], XValidList.shape[1], nFeatures))
XTest = XTestList.reshape((XTestList.shape[0], XTestList.shape[1], nFeatures))

print(XTrain.shape)

Total size of data points for LSTM: 117
Size of training set: 97
Size of validation set: 10
Size of test set: 10
(97, 10, 1)


# Vanilla LSTM

In [11]:
nNeurons = 100
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    
    # define model
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.9417100727249418
Updating best MAPE to 0.9417100727249418...
Updating best seed to 0...
1 0.17646952156801227
Updating best MAPE to 0.17646952156801227...
Updating best seed to 1...
2 99.98502141906049
3 nan
4 99.98755577573996
5 0.24312541697110354
6 0.27232642586099026
7 99.99416535287153
8 0.28091735844920995
9 99.99267628438622
10 99.9847295641452
11 99.98434043584456
12 0.1771833324478238
13 0.18870308468816205
14 1.2643058869666233
15 99.99115183346173
16 0.7868776973619482
17 1.496964123287761
18 99.9861810564904
19 5.684013165951826
20 0.5852176586812449
21 0.5110579153646102
22 0.32501183825145
23 99.9850147529904
24 0.40720076541711553
25 0.21600573105923365
26 0.17680614765999428
27 99.99487357808098
28 0.5653042864690746
29 99.98930500634158
30 0.9357803198421284
31 0.1404994083164046
Updating best MAPE to 0.1404994083164046...
Updating best seed to 31...
32 99.99544773230996
33 1.2360808039089874
34 0.28880741305089136
35 99.98616227228248
36 0.19458713966572344
37 99.

# Stacked LSTM

In [12]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(LSTM(nNeurons, activation='relu', return_sequences=True, input_shape=(nDaysMin, nFeatures)))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 99.99680112316064
Updating best MAPE to 99.99680112316064...
Updating best seed to 0...
1 61.028254793547575
Updating best MAPE to 61.028254793547575...
Updating best seed to 1...
2 40.53551523638252
Updating best MAPE to 40.53551523638252...
Updating best seed to 2...
3 54.53375445460142
4 nan
5 4.0431639598843
Updating best MAPE to 4.0431639598843...
Updating best seed to 5...
6 99.9920204051422
7 99.98614242469607
8 3.863963912023855
Updating best MAPE to 3.863963912023855...
Updating best seed to 8...
9 99.99563113259524
10 99.98846609081993
11 99.98761872877722
12 57.205544921605004
13 0.5881384209516454
Updating best MAPE to 0.5881384209516454...
Updating best seed to 13...
14 0.4206017797442723
Updating best MAPE to 0.4206017797442723...
Updating best seed to 14...
15 0.13185154071487964
Updating best MAPE to 0.13185154071487964...
Updating best seed to 15...
16 55.44256112312988
17 76.15269038662433
18 99.98633243344102
19 99.98680189831391
20 65.47452182730224
21 63.21740963

# Bidirectional LSTM

In [ ]:
# define model
nNeurons = 50
nFeatures = 1

bestValidMAPE = 100
bestSeed = -1
for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrain, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValid, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(Bidirectional(LSTM(nNeurons, activation='relu'), input_shape=(nDaysMin, nFeatures)))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')

# fit model
model.fit(XTrain, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTest, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

0 0.829601093188896
Updating best MAPE to 0.829601093188896...
Updating best seed to 0...
1 99.98532936111594
2 0.10232715145205354
Updating best MAPE to 0.10232715145205354...
Updating best seed to 2...
3 0.268031516637585
4 0.6522140275418556
5 3.774588955041277
6 0.687363851312785
7 0.40160222427786474
8 0.5135348973637752
9 99.98467094443969
10 0.39094096002030815
11 0.4724971922067189
12 0.259167406851866
13 0.41141661266563867
14 1.3645835944140376
15 0.4993591052483512
16 0.3739416686874978
17 1.5043987066884363
18 0.4895200163380804
19 0.3167039349634024
20 1.0216482703573528
21 2.50736153807099
22 0.3339054769275382
23 0.8924107902186337
24 99.984202199214
25 0.23686769523190063
26 9.480523777062826
27 1.030167077037292
28 99.98417640005516
29 0.7753704241173912
30 1.3430546289444982
31 1.416561677346241
32 99.99252079167157
33 0.25147884649033725
34 0.5805596490955482
35 0.4526833937433611
36 0.4690712286269787
37 1.1380632766962822
38 0.6917580854187697
39 1.0858687364745243

# CNN LSTM

In [ ]:
# Number of subsequences to break X into (we do 10 = 5x2, 5 subsequences of size 2 each)
nSeq = 5
nSteps = 2

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainCNN = XTrainList.reshape((XTrainList.shape[0], nSeq, nSteps, nFeatures))
XValidCNN = XValidList.reshape((XValidList.shape[0], nSeq, nSteps, nFeatures))
XTestCNN = XTestList.reshape((XTestList.shape[0], nSeq, nSteps, nFeatures))

# print(XTrainCNN.shape)
# print(XValidCNN.shape)
# print(XTestCNN.shape)

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))
    model.add(LSTM(nNeurons, activation='relu'))
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidCNN, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=nFilters, kernel_size=1, activation='relu'), input_shape=(None, nSteps, nFeatures)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(nNeurons, activation='relu'))
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainCNN, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestCNN, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)

# ConvLSTM

In [ ]:
# Number of subsequences to break X into (we do 10 = 5x2, 5 subsequences of size 2 each)
nSeq = 5
nSteps = 2
# Each input is rows x columns, we have rows=1 and columns=nSteps

# define model
nNeurons = 50
nFeatures = 1
nFilters = 64

bestValidMAPE = 100
bestSeed = -1

# Reshape input
XTrainConv = XTrainList.reshape((XTrainList.shape[0], nSeq, 1, nSteps, nFeatures))
XValidConv = XValidList.reshape((XValidList.shape[0], nSeq, 1, nSteps, nFeatures))
XTestConv = XTestList.reshape((XTestList.shape[0], nSeq, 1, nSteps, nFeatures))

for seed in range(100):
    tf.random.set_seed(seed=seed)
    model = Sequential()
    model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
    model.add(Flatten())
    model.add(Dense(1))
    opt = Adam(learning_rate=0.1)
    model.compile(optimizer=opt, loss='mse')

    # fit model
    model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

    yPred = list(model.predict(XValidConv, verbose=0))
    yPredList = []
    for i in range(len(yPred)):
        yPredList.append(yPred[i][0])

#     for yTrue, yPred in zip(yTest, yPredList):
#         print(yTrue, yPred)

    MAPE = meanAbsolutePercentageError(yValid, yPredList)
    print(seed, MAPE)
    if MAPE < bestValidMAPE:
        print('Updating best MAPE to {}...'.format(MAPE))
        bestValidMAPE = MAPE
        print('Updating best seed to {}...'.format(seed))
        bestSeed = seed

# define model
print('Training model with best seed...')
tf.random.set_seed(seed=bestSeed)
model = Sequential()
model.add(ConvLSTM2D(filters=64, kernel_size=(1,2), activation='relu', input_shape=(nSeq, 1, nSteps, nFeatures)))
model.add(Flatten())
model.add(Dense(1))
opt = Adam(learning_rate=0.1)
model.compile(optimizer=opt, loss='mse')
# fit model
model.fit(XTrainConv, yTrain, epochs=1000, verbose=0)

yPred = list(model.predict(XTestConv, verbose=0))
yPredList = []
for i in range(len(yPred)):
    yPredList.append(yPred[i][0])
    
MAPE = meanAbsolutePercentageError(yTest, yPredList)
print('Test MAPE:', MAPE)
MdSA = medianSymmetricAccuracy(yTest, yPredList)
print('Test MdSA:', MdSA)